<a href="https://colab.research.google.com/github/gamountainmama/data_dames/blob/main/kaggle_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [18]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv("https://gamountainmama.github.io/data_dames/exams.csv")
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group D,some college,standard,completed,59,70,78
1,male,group D,associate's degree,standard,none,96,93,87
2,female,group D,some college,free/reduced,none,57,76,77
3,male,group B,some college,free/reduced,none,70,70,63
4,female,group D,associate's degree,standard,none,83,85,86


In [24]:
# Add column for average score
df['total score'] = df['math score'] + df['reading score'] + df['writing score']
df['success'] = df['total score'].apply(lambda x: 0 if x < 210 else 1)
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,total score,success
0,female,group D,some college,standard,completed,59,70,78,207,0
1,male,group D,associate's degree,standard,none,96,93,87,276,1
2,female,group D,some college,free/reduced,none,57,76,77,210,1
3,male,group B,some college,free/reduced,none,70,70,63,203,0
4,female,group D,associate's degree,standard,none,83,85,86,254,1


In [32]:
df = df.drop(['math score', 'reading score', 'writing score'], axis=1)
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,total score,success
0,female,group D,some college,standard,completed,207,0
1,male,group D,associate's degree,standard,none,276,1
2,female,group D,some college,free/reduced,none,210,1
3,male,group B,some college,free/reduced,none,203,0
4,female,group D,associate's degree,standard,none,254,1


In [33]:
# Determine the number of unique values in each column.
df.dtypes

gender                         object
race/ethnicity                 object
parental level of education    object
lunch                          object
test preparation course        object
total score                     int64
success                         int64
dtype: object

In [34]:
# Convert categorical data to numeric with `pd.get_dummies`
num_data = pd.get_dummies(df)
num_data.head()

,total score,success,gender_female,gender_male,race/ethnicity_group A,race/ethnicity_group B,race/ethnicity_group C,race/ethnicity_group D,race/ethnicity_group E,parental level of education_associate's degree,parental level of education_bachelor's degree,parental level of education_high school,parental level of education_master's degree,parental level of education_some college,parental level of education_some high school,lunch_free/reduced,lunch_standard,test preparation course_completed,test preparation course_none
0,207,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0
1,276,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1
2,210,1,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1
3,203,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1
4,254,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1


In [35]:
# Split our preprocessed data into our features and target arrays
X = num_data.drop('success', axis=1).values
y = num_data['success'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [36]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [37]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input = X_train_scaled.shape[1]
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=input))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 10)                190       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
# Train the model
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("best_model.hdf5", monitor='accuracy', verbose=1,
    save_best_only=True, mode='auto', save_freq=5)

fit_model = nn.fit(X_train_scaled, y_train, epochs=100, verbose=1, callbacks=[checkpoint])

Epoch 1/100
 1/24 [>.............................] - ETA: 16s - loss: 1.2831 - accuracy: 0.3125
Epoch 1: accuracy improved from -inf to 0.39375, saving model to best_model.hdf5
 8/24 [=========>....................] - ETA: 0s - loss: 1.0121 - accuracy: 0.3984 
Epoch 1: accuracy improved from 0.39375 to 0.39687, saving model to best_model.hdf5

Epoch 1: accuracy improved from 0.39687 to 0.40417, saving model to best_model.hdf5
15/24 [=================>............] - ETA: 0s - loss: 0.9768 - accuracy: 0.4042
Epoch 1: accuracy improved from 0.40417 to 0.41719, saving model to best_model.hdf5
24/24 [==============================] - 1s 7ms/step - loss: 0.9401 - accuracy: 0.4187
Epoch 2/100

Epoch 2: accuracy did not improve from 0.41719
 1/24 [>.............................] - ETA: 0s - loss: 0.9663 - accuracy: 0.3750
Epoch 2: accuracy improved from 0.41719 to 0.42708, saving model to best_model.hdf5
 8/24 [=========>....................] - ETA: 0s - loss: 0.9301 - accuracy: 0.4180
Epoch 

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=1)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8/8 [==============================] - 0s 4ms/step - loss: 0.0734 - accuracy: 0.9720
Loss: 0.07336872816085815, Accuracy: 0.972000002861023
